Written from https://www.kaggle.com/aleksandradeis/understanding-clouds-eda

Possible UNet model https://www.kaggle.com/dimitreoliveira/cloud-segmentation-with-utility-scripts-and-keras

We are tasked with multiclass segmentation task: finding 4 different cloud patterns in the images. On the other hand, we make predictions for each pair of image and label separately, so this could be treated as 4 binary segmentation tasks. It is important to notice that images (and masks) are 1400 x 2100, but predicted masks should be 350 x 525.

Multiclass Segmentation:
http://ai.stanford.edu/~sgould/svl/howto/svlMultiClassSegmentation.pdf

In [10]:
import pandas as pd
import os
from glob import glob

In [13]:
# set paths to train and test image datasets
TRAIN_PATH = 'understanding_cloud_organization_384x576/train_images/'
TEST_PATH = 'understanding_cloud_organization_384x576/test_images/'

# load dataframe with train labels
train_df = pd.read_csv('understanding_cloud_organization_384x576/train.csv')
train_fns = sorted(glob(TRAIN_PATH + '*.jpg'))

print('There are {} images in the train set.'.format(len(train_fns)))

# load the filenames for test images
test_fns = sorted(glob(TEST_PATH + '*.jpg'))

print('There are {} images in the test set.'.format(len(test_fns)))

There are 5546 images in the train set.
There are 3698 images in the test set.


In [14]:
train_df.head()

,Image_Label,EncodedPixels
0,0011165.jpg_Fish,20056 257 20440 257 20824 257 21208 257 21592 ...
1,0011165.jpg_Flower,102135 1 102245 275 102629 275 103013 275 1033...
2,0011165.jpg_Gravel,NaN
3,0011165.jpg_Sugar,NaN
4,002be4f.jpg_Fish,17668 241 18051 242 18435 242 18819 242 19203 ...


For each image from the training dataset there are 4 lines for each type of clouds.

Image_Label is a contatenation of the image filename and a cloud type.

If a certain type of clouds in present on the image, the EncodedPixels column is non-null and contains the segmentation map for the corresponding cloud type.

Almost half the rows are null value empty rows.

In [15]:
print('There are {} rows with empty segmentation maps.'.format(len(train_df) - train_df.EncodedPixels.count()))

There are 10348 rows with empty segmentation maps.


In [17]:
# split column
split_df = train_df["Image_Label"].str.split("_", n = 1, expand = True)
# add new columns to train_df
train_df['Image'] = split_df[0]
train_df['Label'] = split_df[1]

# check the result
train_df.head()

fish = train_df[train_df['Label'] == 'Fish'].EncodedPixels.count()
flower = train_df[train_df['Label'] == 'Flower'].EncodedPixels.count()
gravel = train_df[train_df['Label'] == 'Gravel'].EncodedPixels.count()
sugar = train_df[train_df['Label'] == 'Sugar'].EncodedPixels.count()

print('There are {} fish clouds'.format(fish))
print('There are {} flower clouds'.format(flower))
print('There are {} gravel clouds'.format(gravel))
print('There are {} sugar clouds'.format(sugar))

There are 2781 fish clouds
There are 2365 flower clouds
There are 2939 gravel clouds
There are 3751 sugar clouds


In [18]:
labels_per_image = train_df.groupby('Image')['EncodedPixels'].count()
print('The mean number of labels per image is {}'.format(labels_per_image.mean()))

The mean number of labels per image is 2.13415073927


***Exploring the images***

In [19]:
# get sizes of images from test and train sets
train_widths, train_heights, max_train, min_train = get_image_sizes(train = True)
test_widths, test_heights, max_test, min_test = get_image_sizes(train = False)

print('Maximum width for training set is {}'.format(max(train_widths)))
print('Minimum width for training set is {}'.format(min(train_widths)))
print('Maximum height for training set is {}'.format(max(train_heights)))
print('Minimum height for training set is {}'.format(min(train_heights)))

NameError: name 'get_image_sizes' is not defined